Reference:
    https://cirq.readthedocs.io/en/stable/tutorial.html 

In [46]:
import cirq
cirq.__version__

'0.7.0'

## Basics

In [2]:
# define the dimension of the grid
length = 3

In [3]:
# define qubits
qubits = [cirq.GridQubit(i,j) for i in range(length) for j in range(length)]
print(qubits)

[cirq.GridQubit(0, 0), cirq.GridQubit(0, 1), cirq.GridQubit(0, 2), cirq.GridQubit(1, 0), cirq.GridQubit(1, 1), cirq.GridQubit(1, 2), cirq.GridQubit(2, 0), cirq.GridQubit(2, 1), cirq.GridQubit(2, 2)]


In [4]:
# define circuit
circuit = cirq.Circuit()
circuit.append(cirq.H(q) for q in qubits if (q.row + q.col) % 2 == 0)
circuit.append(cirq.X(q) for q in qubits if (q.row + q.col) % 2 == 1)
print(circuit)

(0, 0): ───H───

(0, 1): ───X───

(0, 2): ───H───

(1, 0): ───X───

(1, 1): ───H───

(1, 2): ───X───

(2, 0): ───H───

(2, 1): ───X───

(2, 2): ───H───


In [5]:
# alternative insert strategy
circuit = cirq.Circuit()
circuit.append((cirq.H(q) for q in qubits if (q.row + q.col) % 2 == 0), 
               strategy=cirq.InsertStrategy.EARLIEST)
circuit.append((cirq.X(q) for q in qubits if (q.row + q.col) % 2 == 1), 
               strategy=cirq.InsertStrategy.NEW_THEN_INLINE)
for i, m in enumerate(circuit):
    print('Moment {}: {}'.format(i, m))

Moment 0: H((0, 0)) and H((0, 2)) and H((1, 1)) and H((2, 0)) and H((2, 2))
Moment 1: X((0, 1)) and X((1, 0)) and X((1, 2)) and X((2, 1))


In [6]:
def rot_x_layer(length, half_turns):
    rot = cirq.XPowGate(exponent=half_turns)
    # rot = cos(half_turns * pi) I + i sin(half_turns * pi) X
    for i in range(length):
        for j in range(length):
            yield rot(cirq.GridQubit(i,j))

In [7]:
circuit = cirq.Circuit()
circuit.append(rot_x_layer(2, 0.1))
print(circuit)

(0, 0): ───X^0.1───

(0, 1): ───X^0.1───

(1, 0): ───X^0.1───

(1, 1): ───X^0.1───


## VQE

### Setting up problem definition

In [8]:
import numpy as np

In [9]:
def rand2d(nrows, ncols):
    return np.random.choice([-1, 1], size=(nrows, ncols))

In [10]:
def random_instance(length):
    # transverse field terms
    h = rand2d(length, length)
    # links within a row
    jr = rand2d(length, length-1)
    # links within a column
    jc = rand2d(length-1, length)
    return (h, jr, jc)

In [11]:
h, jr, jc = random_instance(3)

### Setting up quantum circuit

In [12]:
def rot_z_layer(h, half_turns):
    """Yields Z rotations by half_turns conditioned on the field h."""
    gate = cirq.ZPowGate(exponent=half_turns)
    for i, h_row in enumerate(h):
        for j, h_ij in enumerate(h_row):
            if h_ij == 1:
                yield gate(cirq.GridQubit(i, j))

In [13]:
def rot_11_layer(jr, jc, half_turns):
    """Yields rotations about |11> conditioned on the jr and jc fields."""
    gate = cirq.CZPowGate(exponent=half_turns)    
    for i, jr_row in enumerate(jr):
        for j, jr_ij in enumerate(jr_row):
            if jr_ij == -1:
                yield cirq.X(cirq.GridQubit(i, j))
                yield cirq.X(cirq.GridQubit(i, j+1))
                
            yield gate(cirq.GridQubit(i, j),
                       cirq.GridQubit(i, j+1))
            
            if jr_ij == -1:
                yield cirq.X(cirq.GridQubit(i, j))
                yield cirq.X(cirq.GridQubit(i, j+1))

    for i, jc_row in enumerate(jc):
        for j, jc_ij in enumerate(jc_row):
            if jc_ij == -1:
                yield cirq.X(cirq.GridQubit(i, j))
                yield cirq.X(cirq.GridQubit(i+1, j))
            yield gate(cirq.GridQubit(i, j),
                       cirq.GridQubit(i+1, j))
            if jc_ij == -1:
                yield cirq.X(cirq.GridQubit(i, j))
                yield cirq.X(cirq.GridQubit(i+1, j))

In [14]:
def create_ansatz(h, jr, jc, x_half_turns, h_half_turns, j_half_turns):
    yield rot_x_layer(len(h), x_half_turns)
    yield rot_z_layer(h, h_half_turns)
    yield rot_11_layer(jr, jc, j_half_turns)

In [15]:
circuit = cirq.Circuit()
circuit.append(create_ansatz(h, jr, jc, 0.1, 0.2, 0.3))

### Run Simulator

In [16]:
simulator = cirq.Simulator()

In [17]:
%time result = simulator.simulate(circuit)

CPU times: user 16.1 ms, sys: 2.57 ms, total: 18.7 ms
Wall time: 16.8 ms


In [18]:
print(np.around(result.final_state, 3))

[-0.797-0.406j  0.14 +0.022j  0.1  -0.1j   -0.022+0.004j  0.1  +0.1j
 -0.02 -0.01j  -0.02 -0.01j   0.002+0.003j  0.064+0.126j  0.004-0.022j
 -0.022+0.004j  0.001+0.004j -0.004-0.022j -0.002+0.003j  0.002+0.003j
  0.001-0.j     0.14 -0.022j -0.02 +0.01j  -0.02 +0.01j   0.004+0.001j
 -0.022-0.004j  0.004-0.001j  0.003+0.003j -0.   -0.001j -0.004-0.022j
 -0.002+0.003j  0.002+0.003j  0.001-0.j    -0.001+0.004j  0.   -0.j
  0.   -0.j    -0.   -0.j     0.1  -0.1j   -0.01 +0.02j   0.01 +0.02j
  0.001-0.004j -0.02 -0.01j   0.004+0.001j  0.004+0.001j -0.   -0.j
 -0.022+0.004j  0.003+0.002j  0.001-0.004j -0.001-0.j     0.002+0.003j
  0.   -0.001j -0.   -0.j    -0.   +0.j    -0.02 +0.01j   0.003-0.003j
  0.003-0.003j -0.001+0.j     0.003+0.003j -0.001-0.j    -0.   -0.001j
 -0.   +0.j     0.002+0.003j  0.   -0.001j -0.   -0.j    -0.   +0.j
  0.   -0.j    -0.   +0.j    -0.   +0.j     0.   +0.j     0.064+0.126j
 -0.016-0.016j -0.022+0.004j  0.003+0.002j -0.004-0.022j  0.002+0.003j
  0.002+0.003j  0.

In [19]:
circuit.append(cirq.measure(*qubits, key='x'))

In [20]:
%time result = simulator.run(circuit, repetitions=10000)

CPU times: user 157 ms, sys: 2.9 ms, total: 160 ms
Wall time: 161 ms


In [21]:
print(result.histogram(key='x'))

Counter({0: 7975, 8: 232, 64: 228, 4: 208, 128: 199, 256: 198, 2: 197, 32: 197, 16: 189, 1: 186, 80: 9, 20: 8, 160: 8, 33: 7, 144: 7, 130: 7, 96: 7, 68: 7, 34: 7, 320: 7, 6: 6, 72: 6, 36: 6, 9: 6, 129: 5, 48: 5, 260: 5, 12: 5, 264: 5, 136: 5, 384: 5, 288: 5, 18: 5, 24: 4, 192: 4, 5: 4, 132: 4, 66: 4, 258: 4, 257: 3, 10: 3, 272: 3, 40: 2, 17: 2, 65: 2, 164: 1, 42: 1, 88: 1, 41: 1, 14: 1, 81: 1, 3: 1, 276: 1, 336: 1})


In [22]:
def energy_func(h, jr, jc):
    def calc(measurements):
        zs = 1 - 2*measurements.reshape(*h.shape).astype(np.int8)
        tot_energy = np.sum(h * zs)
        for i, jr_row in enumerate(jr):
            for j, jr_ij in enumerate(jr_row):
                tot_energy += jr_ij * zs[i, j] * zs[i, j+1]
        for i, jc_row in enumerate(jc):
            for j, jc_ij in enumerate(jc_row):
                tot_energy += jc_ij * zs[i, j] * zs[i+1, j]
        return tot_energy
    return calc

In [23]:
print(result.histogram(key='x', fold_func=energy_func(h, jr, jc)))

Counter({-7: 8426, -5: 652, 1: 422, -1: 229, -9: 214, -3: 27, 3: 14, 9: 8, 7: 7, 5: 1})


In [24]:
def obj_func(result):
    energy_hist = result.histogram(key='x', fold_func=energy_func(h, jr, jc))
    return np.sum([k * v for k,v in energy_hist.items()]) / result.repetitions

In [27]:
print('Value of the objective function = {}'.format(obj_func(result)))

Value of the objective function = -6.3888


### Parameterize the Ansatz

In [28]:
import sympy
alpha = sympy.Symbol('alpha')
beta = sympy.Symbol('beta')
gamma = sympy.Symbol('gamma')

In [29]:
circuit = cirq.Circuit()
circuit.append(create_ansatz(h, jr, jc, alpha, beta, gamma))
circuit.append(cirq.measure(*qubits, key='x'))

In [30]:
resolver = cirq.ParamResolver({'alpha': 0.1, 'beta': 0.2, 'gamma': 0.3})
resolved_circuit = cirq.resolve_parameters(circuit, resolver)

In [31]:
obj_func(simulator.run(resolved_circuit, repetitions=10000))

-6.3966

### Optimization

In [32]:
from scipy.optimize import minimize
import sys

In [33]:
def fom(ps):
    resolver = cirq.ParamResolver(dict(zip(['alpha', 'beta', 'gamma'], ps)))
    resolved_circuit = cirq.resolve_parameters(circuit, resolver)
    return obj_func(simulator.run(resolved_circuit, repetitions=100000))

In [34]:
def callback(ps):
    print(ps, fom(ps), file=sys.stderr)

In [35]:
%time fom([0.1, 0.2, 0.3])

CPU times: user 9.65 s, sys: 144 ms, total: 9.8 s
Wall time: 9.76 s


-6.38106

In [36]:
res = minimize(
    fun      = fom,
    callback = callback,
    x0       = [0.1, 0.2, 0.3],
    method   = 'Nelder-Mead',
    options  = {'disp'   : 1,
                'maxiter': 10,
                'initial_simplex': [
                    [0.1, 0.2, 0.3],
                    [0.2, 0.2, 0.3],
                    [0.1, 0.3, 0.3],
                    [0.1, 0.2, 0.4],
                ],
                'xatol'  : 1e-2,
                'fatol'  : 3e-3
                }
)

[2.77555756e-17 2.66666667e-01 3.66666667e-01] -7.0
[2.77555756e-17 2.66666667e-01 3.66666667e-01] -7.0
[2.77555756e-17 2.66666667e-01 3.66666667e-01] -7.0
[2.77555756e-17 2.66666667e-01 3.66666667e-01] -7.0
[2.77555756e-17 2.66666667e-01 3.66666667e-01] -7.0
[2.77555756e-17 2.66666667e-01 3.66666667e-01] -7.0
[2.77555756e-17 2.66666667e-01 3.66666667e-01] -7.0
[2.77555756e-17 2.66666667e-01 3.66666667e-01] -7.0


[2.77555756e-17 2.66666667e-01 3.66666667e-01] -7.0


In [37]:
print(res)

 final_simplex: (array([[ 2.77555756e-17,  2.66666667e-01,  3.66666667e-01],
       [ 2.96210562e-03,  2.07580304e-01,  3.67388260e-01],
       [ 3.19144376e-03,  2.40342793e-01,  3.92263232e-01],
       [-7.69032922e-03,  2.13846022e-01,  3.68990055e-01]]), array([-7.     , -6.99918, -6.99898, -6.99596]))
           fun: -7.0
       message: 'Maximum number of iterations has been exceeded.'
          nfev: 20
           nit: 10
        status: 2
       success: False
             x: array([2.77555756e-17, 2.66666667e-01, 3.66666667e-01])


In [38]:
res1 = minimize(
    fun      = fom,
    callback = callback,
    x0       = [0.1, 0.2, 0.3],
    method   = 'L-BFGS-B',
    options  = {'disp'   : 1,
                'maxiter': 10,
                'ftol'   : 3e-3,
                'eps'    : 1e-2
                }
)

[-0.07889059  0.21762684  0.32715408] -6.60812
[-0.00331656  0.22292467  0.32884135] -6.99918
[-0.00331656  0.22292467  0.32884135] -6.99932


In [39]:
res1

      fun: -6.99944
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.236, -0.01 , -0.006])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 44
      nit: 3
   status: 0
  success: True
        x: array([-0.00331656,  0.22292467,  0.32884135])

## Others

### Interfacing with QASM format

In [40]:
from cirq.contrib.qasm_import import circuit_from_qasm
circuit = circuit_from_qasm("""
    OPENQASM 2.0;
    include "qelib1.inc";
    qreg q[3];
    creg meas[3];
    h q;
    measure q -> meas;
    """)
print(circuit)

q_0: ───H───M('meas_0')───

q_1: ───H───M('meas_1')───

q_2: ───H───M('meas_2')───


### Monte Carlo simulations of noise

In [41]:
q = cirq.NamedQubit('a')
circuit = cirq.Circuit(cirq.bit_flip(p=0.2)(q), cirq.measure(q))
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=1000)
print(result.histogram(key='a'))

Counter({0: 772, 1: 228})


In [42]:
q = cirq.NamedQubit('a')
circuit = cirq.Circuit(cirq.H(q), cirq.amplitude_damp(0.2)(q), cirq.measure(q))
simulator = cirq.DensityMatrixSimulator()
result = simulator.run(circuit, repetitions=1000)
print(result.histogram(key='a'))

Counter({0: 587, 1: 413})


### Device

In [43]:
import cirq
from cirq.devices import GridQubit

In [44]:
class Xmon10Device(cirq.Device):
    def __init__(self):
        self.qubits = [GridQubit(i, 0) for i in range(10)]

    def validate_operation(self, operation):
        if not isinstance(operation, cirq.GateOperation):
            raise ValueError('{!r} is not a supported operation'.format(operation))
        if not isinstance(operation.gate, (cirq.CZPowGate,
                                           cirq.XPowGate,
                                           cirq.PhasedXPowGate,
                                           cirq.YPowGate)):
            raise ValueError('{!r} is not a supported gate'.format(operation.gate))
        if len(operation.qubits) == 2:
            p, q = operation.qubits
            if not p.is_adjacent(q):
                raise ValueError('Non-local interaction: {}'.format(repr(operation)))

    def validate_circuit(self, circuit):
        for moment in circuit:
            for operation in moment.operations:
                self.validate_operation(operation)

In [45]:
device = Xmon10Device()
circuit = cirq.Circuit()
circuit.append([cirq.CZ(device.qubits[0], device.qubits[2])])
try: 
    device.validate_circuit(circuit)
except ValueError as e:
    print(e)

Non-local interaction: cirq.CZ.on(cirq.GridQubit(0, 0), cirq.GridQubit(2, 0))
